##Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

##Importing the Dataset

In [2]:
df = pd.read_csv('/content/final_data_RMSSD.csv')
df.drop('ID', axis=1 , inplace=True)

In [3]:
df.head()

,Gender,Age,Alchohol,Smoking,Non-Veg,Exercise,Medication,BMI,rmssd_best
0,Male,37,No,No,No,No,No,24.1,2
1,Male,47,No,No,No,Yes,Yes(Anxiety/B.P.),22.8,2
2,Female,47,No,No,No,No,Yes(Thyroid),28.0,2
3,Female,35,No,No,No,No,No,27.4,2
4,Female,33,No,No,No,No,No,22.6,3


In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy="not majority")
X_res, y_res = ros.fit_resample(X, y)

In [5]:
y_res

0     2
1     2
2     2
3     2
4     3
     ..
79    5
80    5
81    5
82    5
83    5
Name: rmssd_best, Length: 84, dtype: int64

In [6]:
y_res.value_counts()

2    14
3    14
5    14
4    14
1    14
6    14
Name: rmssd_best, dtype: int64

In [7]:
X = X_res.values
y = y_res.values

##LabelEncoding

In [8]:
for i in range(X.shape[0]):
  for j in range(0, 6):
    if X[i, j] == 'No' or X[i,j] == "Male" or X[i, j] == 'No ':
      X[i, j] = 0
    elif X[i, j] == 'Yes' or X[i,j] == "Female":
      X[i, j] = 1

for i in range(X.shape[0]):
  for j in range(6,7):
    if X[i,j] == 'No':
      X[i,j] = 0
    else:
      X[i,j] = 1

##Splitting the Dataset

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
X_train.shape

(67, 8)

##Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.fit_transform(X_test)

#MODELS

In [12]:
# Function to print evaluation metrics
def print_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred) * 100

    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')

    print(f"Accuracy: {accuracy:.10f} %")
    print(f"Precision: {precision_macro:.10f}")
    print(f"Recall: {recall_macro:.10f}")
    print(f"F1 Score: {f1_macro:.10f}")

In [13]:
import warnings
warnings.filterwarnings("ignore")

###Logistic Regression

In [14]:
lr = LogisticRegression()
lr_params = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [50, 100, 200, 500, 1000],
    'class_weight': [None, 'balanced'],
    'random_state': [42]
}
grid_lr = GridSearchCV(lr, lr_params, cv=5)
grid_lr.fit(X_train_scaled, y_train)
y_pred_lr = grid_lr.predict(X_test_scaled)

# Print details of the Logistic Regression model
print("Logistic Regression:")
print("Best Parameters:", grid_lr.best_params_)
print("Best Score:", grid_lr.best_score_* 100 )
print_metrics(y_test, y_pred_lr)
print("\n")

Logistic Regression:
Best Parameters: {'C': 0.1, 'class_weight': None, 'max_iter': 50, 'penalty': 'l2', 'random_state': 42, 'solver': 'liblinear'}
Best Score: 35.604395604395606
Accuracy: 29.4117647059 %
Precision: 0.3611111111
Recall: 0.3194444444
F1 Score: 0.2952380952




In [15]:
import pickle

with open('RMSSD_basic_logistic.pkl', 'wb') as f:
    pickle.dump(grid_lr, f)

##KNeighbors

In [16]:
knn = KNeighborsClassifier()
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 20],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [30, 40, 50, 60],
    'p': [1, 2]
}
grid_knn = GridSearchCV(knn, knn_params, cv=5)
grid_knn.fit(X_train_scaled, y_train)
y_pred_knn = grid_knn.predict(X_test_scaled)

# Print details of the k-Nearest Neighbors model
print("k-Nearest Neighbors:")
print("Best Parameters:", grid_knn.best_params_)
print("Best Score:", grid_knn.best_score_)
print_metrics(y_test, y_pred_knn)
print("\n")

k-Nearest Neighbors:
Best Parameters: {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
Best Score: 0.5252747252747253
Accuracy: 41.1764705882 %
Precision: 0.3888888889
Recall: 0.4444444444
F1 Score: 0.3813492063




In [17]:
with open('RMSSD_basic_KNN.pkl', 'wb') as f:
    pickle.dump(grid_knn, f)

##SVM

In [18]:
svm = SVC()
svm_params = {
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1, 10, 100],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.5, 1.0]
}

grid_svm = GridSearchCV(svm, svm_params, cv=5)
grid_svm.fit(X_train_scaled, y_train)
y_pred_svm = grid_svm.predict(X_test_scaled)

# Print details of the Support Vector Machines model
print("Support Vector Machines:")
print("Best Parameters:", grid_svm.best_params_)
print("Best Score:", grid_svm.best_score_)
print_metrics(y_test, y_pred_svm)
print("\n")


Support Vector Machines:
Best Parameters: {'C': 10, 'coef0': 0.0, 'degree': 2, 'gamma': 10, 'kernel': 'rbf'}
Best Score: 0.5527472527472528
Accuracy: 29.4117647059 %
Precision: 0.2857142857
Recall: 0.2361111111
F1 Score: 0.2055555556




In [19]:
with open('RMSSD_basic_SVM.pkl', 'wb') as f:
    pickle.dump(grid_svm, f)

##Decision Trees

In [20]:
dt = DecisionTreeClassifier()
dt_params = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20, 30, 50, 100],
    'min_samples_split': [2, 5, 10, 20, 30, 50],
    'min_samples_leaf': [1, 2, 4, 8, 10, 20]
}

grid_dt = GridSearchCV(dt, dt_params, cv=5)
grid_dt.fit(X_train, y_train)
y_pred_dt = grid_dt.predict(X_test)

# Print details of the Decision Trees model
print("Decision Trees:")
print("Best Parameters:", grid_dt.best_params_)
print("Best Score:", grid_dt.best_score_)
print_metrics(y_test, y_pred_dt)
print("\n")


Decision Trees:
Best Parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
Best Score: 0.5681318681318681
Accuracy: 64.7058823529 %
Precision: 0.6805555556
Recall: 0.7083333333
F1 Score: 0.6380952381




In [21]:
with open('RMSSD_basic_DecTree.pkl', 'wb') as f:
    pickle.dump(grid_dt, f)

##Random Forest

In [22]:
rf = RandomForestClassifier()
rf_params = {
    'n_estimators': [50, 100, 200, 400, 800],
    'criterion': ['gini'],
    'max_depth': [None, 5, 10, 20,50],
    'min_samples_leaf': [1, 2, 4, 8]
}
grid_rf = GridSearchCV(rf, rf_params, cv=5)
grid_rf.fit(X_train, y_train)
y_pred_rf = grid_rf.predict(X_test)

# Print details of the Random Forest model
print("Random Forest:")
print("Best Parameters:", grid_rf.best_params_)
print("Best Score:", grid_rf.best_score_)
print_metrics(y_test, y_pred_rf)
print("\n")

Random Forest:
Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 50}
Best Score: 0.5406593406593407
Accuracy: 70.5882352941 %
Precision: 0.7666666667
Recall: 0.7500000000
F1 Score: 0.7361111111




In [23]:
with open('RMSSD_basic_RanFor.pkl', 'wb') as f:
    pickle.dump(grid_rf, f)

##NaiveBayes

In [24]:
gnb = GaussianNB()
gnb_params = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  # Varying the smoothing parameter
}
grid_gnb = GridSearchCV(gnb, gnb_params, cv=5)
grid_gnb.fit(X_train, y_train)

y_pred_gnb = grid_gnb.predict(X_test)

print("Gaussian Naive Bayes:")
print("Best Parameters:", grid_gnb.best_params_)
print("Best Score:", grid_gnb.best_score_)
print_metrics(y_test, y_pred_gnb)
print("\n")


Gaussian Naive Bayes:
Best Parameters: {'var_smoothing': 1e-09}
Best Score: 0.3582417582417582
Accuracy: 35.2941176471 %
Precision: 0.2592592593
Recall: 0.3888888889
F1 Score: 0.2962962963




In [25]:
with open('RMSSD_basic_NaiveBayes.pkl', 'wb') as f:
    pickle.dump(grid_gnb, f)